In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline


In [3]:
train = pd.read_csv('hacktrain.csv')
test = pd.read_csv('hacktest.csv')

In [18]:
ndvi_cols =[col for col in train.columns if col.endswith('_N')]

In [26]:
def add_feat(df):
    df_feat = df.copy()
    df_feat['ndvi_mean'] = df[ndvi_cols].mean(axis=1)
    df_feat['ndvi_std'] = df[ndvi_cols].std(axis=1)
    df_feat['ndvi_min'] = df[ndvi_cols].min(axis=1)
    df_feat['ndvi_max'] = df[ndvi_cols].max(axis=1)
    df_feat['ndvi_range'] = df_feat['ndvi_max'] - df_feat['ndvi_min']
    return df_feat

In [100]:
x_train=train_feat[ndvi_cols+['ndvi_mean', 'ndvi_std', 'ndvi_min', 'ndvi_max', 'ndvi_range']]
x_test=test_feat[ndvi_cols+['ndvi_mean', 'ndvi_std', 'ndvi_min', 'ndvi_max', 'ndvi_range']]
y_train=train['class']

In [34]:
label_encoder=LabelEncoder()
y_train_encoded=label_encoder.fit_transform(y_train)

In [54]:
x_train_imputed =x_train.T.fillna(x_train.mean(axis=1)).T

In [56]:
x_train_imputed

,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,20150226_N,20150210_N,20150125_N,...,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N,ndvi_mean,ndvi_std,ndvi_min,ndvi_max,ndvi_range
0,637.5950,658.668000,-1882.030000,-1924.360000,997.90400,-1739.990000,630.087000,-189.082283,-1325.640000,-944.08400,...,-189.082283,211.328000,-2203.020,-1180.190000,433.906,-356.566818,1100.007185,-2203.020,997.904,3200.924
1,634.2400,593.705000,-1625.790000,-1672.320000,914.19800,-692.386000,707.626000,-1670.590000,-1408.640000,-989.28500,...,476.972000,220.878000,-2250.000,-1360.560000,524.075,-252.624381,953.792308,-2250.000,914.198,3164.198
2,58.0174,-1599.160000,-298.814680,-1052.630000,-298.81468,-1564.630000,-298.814680,729.790000,-3162.140000,-298.81468,...,-298.814680,369.214000,-2202.120,-298.814680,-1343.550,-511.704400,1158.179688,-3162.140,952.178,4114.318
3,72.5180,-128.956231,380.436000,-1256.930000,515.80500,-1413.180000,-802.942000,683.254000,-2829.400000,-1267.54000,...,282.833000,298.320000,-2197.360,-128.956231,-826.727,-290.295917,1053.427212,-2829.400,1232.120,4061.520
4,1136.4400,683.818913,683.818913,1647.830000,1935.80000,683.818913,2158.980000,683.818913,1242.870000,-2646.05000,...,1020.720000,1660.650000,-116.801,-568.050000,-1357.140,553.286190,1289.035547,-2646.050,2158.980,4805.030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1207.7000,984.620000,915.488510,1166.250000,937.47800,1072.700000,823.896000,943.698000,944.619000,1040.99000,...,465.843000,362.882000,979.795,915.488510,433.659,922.351818,326.097943,362.882,1589.000,1226.118
7996,2170.3500,1419.720000,1361.000000,1478.710000,983.91100,1262.110000,1422.860000,1401.970000,1372.680000,1142.34000,...,485.204000,446.724000,771.747,1589.060000,506.936,1248.993808,557.840451,336.289,2178.670,1842.381
7997,1895.6800,1454.740000,1561.345686,1033.560000,1930.38000,1057.150000,1471.600000,1561.345686,2693.630000,2377.55000,...,1170.880000,1095.000000,1818.650,2501.720000,1247.770,1563.116917,639.241970,447.868,2780.930,2333.062
7998,3465.7400,1283.320000,413.412000,4391.050000,1146.82000,4473.050000,1614.750000,770.469000,2465.101495,2676.00000,...,744.143000,2465.101495,3759.710,2465.101495,388.346,2318.030304,1722.594557,244.594,5833.760,5589.166


In [72]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, solver='lbfgs'))
])

In [74]:
pipeline.fit(x_train_imputed, y_train_encoded)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', LogisticRegression(max_iter=1000))])

In [78]:
y_test_encoded = pipeline.predict(x_test)
y_test = label_encoder.inverse_transform(y_test_encoded)

In [84]:
submission = pd.DataFrame({
    'ID': test['ID'],
    'class': y_test
})

In [98]:
submission.to_csv('submission.csv', index=False)